In [1]:
import pandas as pd
import sqlite3
import os
import shutil
from datetime import datetime
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import find_dotenv, load_dotenv

# Update des données  / archivages des fichiers sqlite3
TERMINE


In [33]:

df = pd.read_parquet('data_sources_from_python\df_book.parquet')
df.tail(10)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\chric\AppData\Local\Temp\ipykernel_188\86564744.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_parquet('data_sources_from_python\df_book.parquet')


,id,Titre,Auteurs,notes,Date dernière ouverture,page,Série,Langue,total_read_time,total Nbr de pages lues,id long,Auteurs courts,format,categorie
62,63,Blackwater 1 - La crue: L'épique saga de la fa...,Michael McDowell,0,2024-08-18,248,Blackwater #1,fr,9673.0,234,00063,Michael McDowell,ebook,fiction/fantastique
63,64,Blackwater 2 - La digue,Michael McDowell,0,2024-08-20,225,Blackwater #2,it,9457.0,208,00064,Michael McDowell,ebook,fiction/fantastique
64,65,Le livre des choses perdues,John Connolly,0,2024-09-09,480,N/A,fr,21326.0,467,00065,John Connolly,ebook,fiction/fantasy
65,66,Histoires de moine et de robot T2 : Une prière...,Becky Chambers,0,2024-09-23,139,Histoires de moine et de robot #2,fr,5787.0,135,00066,Becky Chambers,ebook,fiction/SF
66,67,Voyageur 1 : L'Espace d'un an,Becky Chambers,0,2024-09-29,789,Les voyageurs #1,fr,26888.0,779,00067,Becky Chambers,ebook,fiction/SF
67,68,Voyageur 2 : Libration,Becky Chambers,0,2024-10-03,504,Les voyageurs #2,fr,21135.0,487,00068,Becky Chambers,ebook,fiction/SF
68,69,Blackwater 3 – La Maison: L'épique saga de la ...,Michael McDowell,0,2024-10-08,213,Blackwater #3,fr,8572.0,206,00069,Michael McDowell,ebook,fiction/fantastique
69,70,Voyageur 3 : Archives de l'exode,Becky Chambers,0,2024-10-23,441,Les voyageurs #3,fr,22254.0,422,00070,Becky Chambers,ebook,fiction/SF
70,71,Blackwater 4 – La Guerre: L'épique saga de la ...,Michael McDowell,0,2024-10-27,257,Blackwater #4,fr,8562.0,249,00071,Michael McDowell,ebook,fiction/fantastique
71,72,Voyageur 4 : La Galaxie vue du sol,Becky Chambers,0,2024-11-01,400,Les voyageurs #4,fr,2951.0,54,00072,Becky Chambers,ebook,fiction/SF


In [15]:
# load env
load_dotenv()
OPENAI_API_VERSION = os.environ["AZURE_OPENAI_API_VERSION"]
AZURE_OPENAI_ENDPOINT= os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_API_KEY = os.environ["AZURE_OPENAI_API_KEY"]

llm_gpt4o = AzureChatOpenAI(deployment_name="gpt4o",api_key=AZURE_OPENAI_API_KEY,api_version= "2024-02-01",temperature=0)

prompt = """
en utilisant tes connaissances, classe l'oeuvre dans une des catégories suivantes : 

fiction/SF
fiction/fantasy
fiction/fantastique

non-fiction/lifestyle
non-fiction/parenting
non-fiction/histoire
non-fiction/santé

si aucune des catégories ne convient, tu peux en générer une autre, en gardant la même structure (en commençant toujours par fiction ou non-fiction).
n'ajoute rien d'autre.

exemples : 
harry potter et la chambre des secrets - JK Rowling = fiction/fantastique
Blackwater 1 - La crue - Michael McDowell = fiction/fantastique

question: {oeuvre} 

catégorie:"""
rag_prompt_1 = PromptTemplate.from_template(prompt)

# langchain :
chain_category = (
    {"oeuvre": RunnablePassthrough()}
    | rag_prompt_1
    | llm_gpt4o
    | StrOutputParser()
)

	
# fonction pour créer une colonne catégorie
def find_category(oeuvre):
    result = chain_category.invoke(oeuvre)
    return result

In [32]:
df["oeuvre"] = df["Titre"] + " - " + df["Auteurs"]

In [27]:
# Applique find_category uniquement si 'categorie' est vide

df['categorie'] = df.apply(
    lambda row: find_category(row['oeuvre']) if row['categorie'] == '' else row['categorie'], axis=1
)



In [28]:
df.head()

,id,Titre,Auteurs,notes,Date dernière ouverture,page,Série,Langue,total_read_time,total Nbr de pages lues,id long,Auteurs courts,format,categorie,oeuvre
0,1,KOReader Quickstart Guide,N/A,0,2023-11-21,8,N/A,N/A,54.0,4,00001,N/A,ebook,non-fiction/guide,KOReader Quickstart Guide - N/A
1,2,Le crime de l'Orient-Express,"Christie, Agatha",0,2023-11-27,349,l'Orient #1,fr,13191.0,330,00002,Christie Agatha,ebook,fiction/SF,"Le crime de l'Orient-Express - Christie, Agatha"
2,3,Let's Get Together,Isaac Asimov,0,2023-11-27,68,N/A,en,388.0,8,00003,Isaac Asimov,ebook,fiction/SF,Let's Get Together - Isaac Asimov
3,4,La nuit des temps,"Barjavel,René",0,2024-01-08,449,N/A,und,12189.0,288,00004,Barjavel René,ebook,fiction/SF,"La nuit des temps - Barjavel,René"
4,5,"Chasseur, cueilleur, parent: les cultures ance...",Michaeleen Doucleff,0,2024-01-09,675,N/A,fr,18029.0,418,00005,Michaeleen Doucleff,ebook,non-fiction/parenting,"Chasseur, cueilleur, parent: les cultures ance..."


In [34]:
# drop la colonne categorie, puis recréé la vide
df = df.drop(columns=['categorie'])

df.tail()

,id,Titre,Auteurs,notes,Date dernière ouverture,page,Série,Langue,total_read_time,total Nbr de pages lues,id long,Auteurs courts,format
67,68,Voyageur 2 : Libration,Becky Chambers,0,2024-10-03,504,Les voyageurs #2,fr,21135.0,487,00068,Becky Chambers,ebook
68,69,Blackwater 3 – La Maison: L'épique saga de la ...,Michael McDowell,0,2024-10-08,213,Blackwater #3,fr,8572.0,206,00069,Michael McDowell,ebook
69,70,Voyageur 3 : Archives de l'exode,Becky Chambers,0,2024-10-23,441,Les voyageurs #3,fr,22254.0,422,00070,Becky Chambers,ebook
70,71,Blackwater 4 – La Guerre: L'épique saga de la ...,Michael McDowell,0,2024-10-27,257,Blackwater #4,fr,8562.0,249,00071,Michael McDowell,ebook
71,72,Voyageur 4 : La Galaxie vue du sol,Becky Chambers,0,2024-11-01,400,Les voyageurs #4,fr,2951.0,54,00072,Becky Chambers,ebook


In [35]:
# enregistre le dataframe en df_book_cleaned.parquet
df.to_parquet('data_sources_from_python\df_book.parquet', engine='pyarrow')

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\chric\AppData\Local\Temp\ipykernel_188\3345659750.py:2: SyntaxWarning: invalid escape sequence '\d'
  df.to_parquet('data_sources_from_python\df_book.parquet', engine='pyarrow')


élément de langage : 

* df_book_stale = df book déjà existant, provient du ficher parquet déjà existant
* df_book_new = table book du sqlite qui provient du dossier "update",

In [9]:
# df_book_stale
df_book_stale_source = pd.read_parquet("data_sources_from_python/df_book.parquet")
df_book_stale = df_book_stale_source

# drop les lignes 10 à la fin, pour simuler des nouvelles données   
df_book_stale = df_book_stale.drop(df_book_stale.index[10:])
df_book_stale.head(15)
print(df_book_stale.columns)


Index(['id', 'Titre', 'Auteurs', 'notes', 'Date dernière ouverture', 'page',
       'Série', 'Langue', 'total_read_time', 'total Nbr de pages lues',
       'oeuvre', 'id long', 'Auteurs courts', 'format', 'categorie'],
      dtype='object')


In [24]:
# df_book_new

# Chemin vers SQLite
database_path = 'sqlite/statistics.sqlite3'

# Connexion à la base de données SQLite
conn = sqlite3.connect(database_path)

# Liste toutes les tables de la base de données
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)

# Charger une table de la base de données dans un df pandas
book = 'book'
df_book_new = pd.read_sql(f"SELECT * FROM {book};", conn)

# fermer de la connexion à la base de données
conn.close()

# enrichissement du fichier source
df_book_new['oeuvre'] = df_book_new[['title', 'authors']].astype(str).agg(' - '.join, axis=1)
df_book_new['id long'] = df_book_new['id'].apply(lambda x: str(x).zfill(5))
# renaming des colonnes
df_book_new = df_book_new.rename(
    columns={'last_open':'Date dernière ouverture',
             "title":"Titre",
             "authors":"Auteurs",
             "series":"Série",
             "language":"Langue",
             "total_read_pages":"total Nbr de pages lues",
             "pages":"page"})
df_book_new['Date dernière ouverture'] = pd.to_datetime(df_book_new['Date dernière ouverture'], unit='s').dt.date
# créer la colonne "auteur courts"
df_book_new['Auteurs courts'] = (
    df_book_new['Auteurs']
    .str.replace(r'[.,]', ' ', regex=True)  # Remplace les points et virgules par des espaces
    .str.split()
    .apply(lambda x: ' '.join([word for word in x if len(word) > 2]))  # Garde les mots de plus de 2 caractères
    .str.split()
    .apply(lambda x: ' '.join(x[:2]))  # Garde les deux premiers mots
)

# ajout  colonne "format" = "ebook"
df_book_new['format'] = "ebook"
# ajout d'une colonne categorie vide 
df_book_new['categorie'] = ''
df_book_new = df_book_new.drop(columns=['highlights', 'md5'])

In [25]:
print(df_book_new.columns)

Index(['id', 'Titre', 'Auteurs', 'notes', 'Date dernière ouverture', 'page',
       'Série', 'Langue', 'total_read_time', 'total Nbr de pages lues',
       'oeuvre', 'id long', 'Auteurs courts', 'format', 'categorie'],
      dtype='object')


In [45]:
# ajouter 4 à la colonne "total Nbr de pages lues" pour les lignes 5 à 9
df_book_new.loc[5:9, 'total Nbr de pages lues'] += 4
df_book_new.iloc[5:9]

,id,Titre,Auteurs,notes,Date dernière ouverture,page,Série,Langue,total_read_time,total Nbr de pages lues,oeuvre,id long,Auteurs courts,format,categorie
5,6,2023-12-31-23-59-40-Chasseur_ cueilleur_ paren...,N/A,0,2023-12-31,2,N/A,N/A,24.0,49,2023-12-31-23-59-40-Chasseur_ cueilleur_ paren...,00006,N/A,ebook,
6,7,2024-01-01-12-01-52-Chasseur_ cueilleur_ paren...,N/A,0,2024-01-05,36,N/A,N/A,890.0,52,2024-01-01-12-01-52-Chasseur_ cueilleur_ paren...,00007,N/A,ebook,
7,8,What to Expect the First Year,Arlene Eisenberg\nSharon Mazel\nSandee Hathawa...,0,2024-01-08,2473,N/A,en,289.0,75,What to Expect the First Year - Arlene Eisenbe...,00008,Arlene Eisenberg,ebook,
8,9,koreader,N/A,0,2024-01-08,1,N/A,N/A,NaN,48,koreader - N/A,00009,N/A,ebook,


In [46]:
# df_book_stale est la table originale et df_book_new est la table mise à jour

# Comparer les 'id' pour identifier les nouvelles lignes
df_increment = df_book_new[~df_book_new['id'].isin(df_book_stale['id'])]

# Ajouter les nouvelles lignes à df_book_stale
df_book_stale_concatenated = pd.concat([df_book_stale, df_increment], ignore_index=True)


In [47]:
print(df_book_stale_concatenated.shape)
df_book_stale_concatenated.head(15)

(70, 15)


,id,Titre,Auteurs,notes,Date dernière ouverture,page,Série,Langue,total_read_time,total Nbr de pages lues,oeuvre,id long,Auteurs courts,format,categorie
0,1,KOReader Quickstart Guide,N/A,0,2023-11-21,8,N/A,N/A,54.0,4,KOReader Quickstart Guide - N/A,00001,N/A,ebook,non-fiction/guide
1,2,Le crime de l'Orient-Express,"Christie, Agatha",0,2023-11-27,349,l'Orient #1,fr,13191.0,330,"Le crime de l'Orient-Express - Christie, Agatha",00002,Christie Agatha,ebook,fiction/policier
2,3,Let's Get Together,Isaac Asimov,0,2023-11-27,68,N/A,en,388.0,8,Let's Get Together - Isaac Asimov,00003,Isaac Asimov,ebook,fiction/SF
3,4,La nuit des temps,"Barjavel,René",0,2024-01-08,449,N/A,und,12189.0,288,"La nuit des temps - Barjavel,René",00004,Barjavel René,ebook,fiction/SF
4,5,"Chasseur, cueilleur, parent: les cultures ance...",Michaeleen Doucleff,0,2024-01-09,675,N/A,fr,18029.0,418,"Chasseur, cueilleur, parent: les cultures ance...",00005,Michaeleen Doucleff,ebook,non-fiction/parenting
5,6,2023-12-31-23-59-40-Chasseur_ cueilleur_ paren...,N/A,0,2023-12-31,2,N/A,N/A,24.0,1,2023-12-31-23-59-40-Chasseur_ cueilleur_ paren...,00006,N/A,ebook,non-fiction/parenting
6,7,2024-01-01-12-01-52-Chasseur_ cueilleur_ paren...,N/A,0,2024-01-05,36,N/A,N/A,890.0,4,2024-01-01-12-01-52-Chasseur_ cueilleur_ paren...,00007,N/A,ebook,non-fiction/parenting
7,8,What to Expect the First Year,Arlene Eisenberg\nSharon Mazel\nSandee Hathawa...,0,2024-01-08,2473,N/A,en,289.0,27,What to Expect the First Year - Arlene Eisenbe...,00008,Arlene Eisenberg,ebook,non-fiction/parenting
8,9,koreader,N/A,0,2024-01-08,1,N/A,N/A,NaN,0,koreader - N/A,00009,N/A,ebook,non-fiction/technologie
9,10,Black Destroyer,A. E. Van Vogt,0,2024-01-08,60,N/A,en,14.0,1,Black Destroyer - A. E. Van Vogt,00010,Van Vogt,ebook,fiction/SF


In [48]:
# Colonnes à mettre à jour
update_columns = ['notes', 'Date dernière ouverture', 'total_read_time', 'total Nbr de pages lues']

# Faire la jointure sur 'id' pour obtenir les lignes correspondantes de df_fake
df_updated = df_book_stale_concatenated.merge(df_book_new[['id'] + update_columns], on='id', how='left', suffixes=('', '_fake'))

# Remplacer les valeurs existantes par les valeurs de df_fake si elles existent
for col in update_columns:
    df_updated[col] = df_updated[f'{col}_fake'].combine_first(df_updated[col])

# Supprimer les colonnes temporaires (celles avec le suffixe "_fake")
df_updated.drop(columns=[f'{col}_fake' for col in update_columns], inplace=True)



In [50]:


# Ajout de la colonne catégorie mais uniquement sur les lignes vides : 

# AJOUT DE LA CATEGORY : 

# load env
load_dotenv()
OPENAI_API_VERSION = os.environ["AZURE_OPENAI_API_VERSION"]
AZURE_OPENAI_ENDPOINT= os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_API_KEY = os.environ["AZURE_OPENAI_API_KEY"]

llm_gpt4o = AzureChatOpenAI(deployment_name="gpt4o",api_key=AZURE_OPENAI_API_KEY,api_version= "2024-02-01",temperature=0)

prompt = """
en utilisant tes connaissances, classe l'oeuvre dans une des catégories suivantes : 

fiction/SF
fiction/fantasy
fiction/fantastique

non-fiction/lifestyle
non-fiction/parenting
non-fiction/histoire
non-fiction/santé

si aucune des catégories ne convient, tu peux en générer une autre, en gardant la même structure (en commençant toujours par fiction ou non-fiction).
n'ajoute rien d'autre.

exemples : 
harry potter et la chambre des secrets - JK Rowling = fiction/fantastique
Blackwater 1 - La crue - Michael McDowell = fiction/fantastique

question: {oeuvre} 

catégorie:"""
rag_prompt_1 = PromptTemplate.from_template(prompt)

# langchain :
chain_category = (
    {"oeuvre": RunnablePassthrough()}
    | rag_prompt_1
    | llm_gpt4o
    | StrOutputParser()
)

	
# fonction pour créer une colonne catégorie
def find_category(oeuvre):
    result = chain_category.invoke(oeuvre)
    return result

df_updated['categorie'] = df_updated['oeuvre'].apply(find_category)
df_updated = df_updated.drop(columns=['oeuvre'])


Nouvelle version fonctionnelle !

Derniere étape : le déplacement du fichier sqlite

In [148]:
# fonctionne avec 3 dossiers, fonctionne.
# hypothèse en cours, utiliser la version 2 dossiers
#  un dossier de mise à jour, un dossier d'archive (contenant la version de référence) et un sous dossier old/archive



# Définir les chemins des dossiers et du fichier
folder_update_path = "sqlite/update"
folder_archive_path = "sqlite/archive"
folder_old_archive_path = "sqlite/archive/old"
sqlite_update_path = os.path.join(folder_update_path, "statistics.sqlite3")

# Vérifier si le fichier existe dans le dossier de mise à jour
if os.path.exists(sqlite_update_path):
    # Vérifier si le dossier old existe, sinon le créer
    if not os.path.exists(folder_old_archive_path):
        os.makedirs(folder_old_archive_path)

    # Déplacer le fichier actuel dans archive vers old
    archive_files = [f for f in os.listdir(folder_archive_path) if os.path.isfile(os.path.join(folder_archive_path, f))]

    if archive_files:
        latest_archive_file = archive_files[0]  # Le fichier actuel dans archive
        shutil.move(os.path.join(folder_archive_path, latest_archive_file), os.path.join(folder_old_archive_path, latest_archive_file))
        print(f"le fichier initial {latest_archive_file} a été déplacé vers {folder_old_archive_path}")

    # Générer un horodatage dans le format souhaité (exemple : YYYYMMDD_HHMMSS)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Construire le nouveau nom de fichier avec l'horodatage
    new_filename = f"statistics_{timestamp}.sqlite3"
    new_filepath = os.path.join(folder_archive_path, new_filename)

    # Déplacer et renommer le fichier vers le dossier archive
    shutil.move(sqlite_update_path, new_filepath)
    print(f"le fichier de mise à jour a été déplacé vers archive et renommé en {new_filename}")

    # Compter le nombre de fichiers dans le dossier archive/old
    num_files = len([name for name in os.listdir(folder_old_archive_path) if os.path.isfile(os.path.join(folder_old_archive_path, name))])
    if num_files > 3:
        # Afficher le message avec le nombre de fichiers
        print(f"Il y a {num_files} fichiers de sauvegarde dans le dossier old, pense à faire du ménage, tu vas pas garder 2000 sauvegardes, tu payes pas tes bytes ou quoi ?")

else:
    print(f"Aucun fichier trouvé à l'emplacement : {sqlite_update_path}")

le fichier de mise à jour a été déplacé vers archive et renommé en statistics_20241101_163632.sqlite3
Il y a 6 fichiers de sauvegarde dans le dossier old, pense à faire du ménage, tu vas pas garder 2000 sauvegardes, tu payes pas tes bytes ou quoi ?


In [ ]:

# VERSION avec 2 dossiers, fonctionne

import os
import shutil
from datetime import datetime

# Définir les chemins des dossiers et du fichier
folder_update_path = "sqlite/update"
folder_archive_path = "sqlite/archive"
sqlite_update_path = os.path.join(folder_update_path, "statistics.sqlite3")

# Vérifier si le fichier existe dans le dossier de mise à jour
if os.path.exists(sqlite_update_path):
    # Générer un horodatage dans le format souhaité (exemple : YYYYMMDD_HHMMSS)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Construire le nouveau nom de fichier avec l'horodatage
    new_filename = f"statistics_{timestamp}.sqlite3"
    new_filepath = os.path.join(folder_archive_path, new_filename)
    
    # Déplacer et renommer le fichier vers le dossier archive
    shutil.move(sqlite_update_path, new_filepath)
    print(f"Fichier déplacé et renommé en {new_filename}")
    # Compter le nombre de fichiers dans le dossier archive
    num_files = len([name for name in os.listdir(folder_archive_path) if os.path.isfile(os.path.join(folder_archive_path, name))])
    if num_files > 3:
        # Afficher le message avec le nombre de fichiers
        print(f"Il y a {num_files} fichiers de sauvegarde, pense à faire du ménage, tu vas pas garder 2000 sauvegardes, tu payes pas tes bytes ou quoi ?")

    
else:
    print(f"Aucun fichier trouvé à l'emplacement : {sqlite_update_path}")



# nouvelle colonne genAI
EN COURS

In [ ]:
# modèle
prompt = """
en utilisant tes connaissances, classe l'oeuvre dans une des catégories suivantes : 

fiction/SF
fiction/fantasy
fiction/fantastique

non-fiction/lifestyle
non-fiction/parenting
non-fiction/histoire
non-fiction/santé

si aucune des catégories ne convient, tu peux en générer une autre, en gardant la même structure (en commençant toujours par fiction ou non-fiction).
n'ajoute rien d'autre.

exemples : 
harry potter et la chambre des secrets - JK Rowling = fiction/fantastique
Blackwater 1 - La crue - Michael McDowell = fiction/fantastique

question: {oeuvre} 

catégorie:"""
rag_prompt_1 = PromptTemplate.from_template(prompt)

# langchain :
chain_category = (
    {"oeuvre": RunnablePassthrough()}
    | rag_prompt_1
    | llm_gpt4o
    | StrOutputParser()
)

	
# fonction pour créer une colonne catégorie
def find_category(oeuvre):
    result = chain_category.invoke(oeuvre)
    return result



df_updated['categorie'] = df_updated.apply(
    lambda row: find_category(row['oeuvre']) if pd.isna(row['categorie']) or row['categorie'] == '' else row['categorie'],
    axis=1
)

In [44]:

df = pd.read_parquet('data_sources_from_python\df_book.parquet')
df.tail(10)
# garde uniquement les 5 dernières lignes
df = df.tail(5)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\chric\AppData\Local\Temp\ipykernel_188\3317538426.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_parquet('data_sources_from_python\df_book.parquet')


### Trouver la date de première édition en version originale
TERMINE

In [45]:
prompt_annee_edition = """
en utilisant tes connaissances, trouve la date de première parution de l'oeuvre dans sa version originale. (parfois appelée First published date).
N'ajoute rien d'autre.
Le format doit être AAAA.

question: {oeuvre} 

catégorie:"""
prompt = PromptTemplate.from_template(prompt_annee_edition)

# langchain :
chain_first_published_date = (
    {"oeuvre": RunnablePassthrough()}
    | prompt
    | llm_gpt4o
    | StrOutputParser()
)

	
# fonction pour créer une colonne catégorie
def first_published_date(oeuvre):
    result = chain_first_published_date.invoke(oeuvre)
    return result


if 'First published date' not in df.columns:
    df['First published date'] = ''
df['First published date'] = df.apply(
    lambda row: first_published_date(row['oeuvre']) if pd.isna(row['First published date']) or row['First published date'] == '' else row['First published date'],
    axis=1
)

In [46]:
df.tail()

,id,Titre,Auteurs,notes,Date dernière ouverture,page,Série,Langue,total_read_time,total Nbr de pages lues,id long,Auteurs courts,format,categorie,oeuvre,First published date
67,68,Voyageur 2 : Libration,Becky Chambers,0,2024-10-03,504,Les voyageurs #2,fr,21135.0,487,00068,Becky Chambers,ebook,fiction/SF,Voyageur 2 : Libration - Becky Chambers,2022
68,69,Blackwater 3 – La Maison: L'épique saga de la ...,Michael McDowell,0,2024-10-08,213,Blackwater #3,fr,8572.0,206,00069,Michael McDowell,ebook,fiction/fantastique,Blackwater 3 – La Maison: L'épique saga de la ...,1983
69,70,Voyageur 3 : Archives de l'exode,Becky Chambers,0,2024-10-23,441,Les voyageurs #3,fr,22254.0,422,00070,Becky Chambers,ebook,fiction/SF,Voyageur 3 : Archives de l'exode - Becky Chambers,2018
70,71,Blackwater 4 – La Guerre: L'épique saga de la ...,Michael McDowell,0,2024-10-27,257,Blackwater #4,fr,8562.0,249,00071,Michael McDowell,ebook,fiction/fantastique,Blackwater 4 – La Guerre: L'épique saga de la ...,1983
71,72,Voyageur 4 : La Galaxie vue du sol,Becky Chambers,0,2024-11-01,400,Les voyageurs #4,fr,2951.0,54,00072,Becky Chambers,ebook,fiction/SF,Voyageur 4 : La Galaxie vue du sol - Becky Cha...,2021


In [43]:
df.to_parquet('data_sources_from_python\df_annee.parquet', engine='pyarrow')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\chric\AppData\Local\Temp\ipykernel_188\3199247240.py:1: SyntaxWarning: invalid escape sequence '\d'
  df.to_parquet('data_sources_from_python\df_annee.parquet', engine='pyarrow')


### genai
EN COURS

In [47]:
df = pd.read_parquet('data_sources_from_python\df_book.parquet')
df.tail(10)
# garde uniquement les 5 dernières lignes
df = df.tail(5)
df.head(20)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\chric\AppData\Local\Temp\ipykernel_188\3727004748.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_parquet('data_sources_from_python\df_book.parquet')


,id,Titre,Auteurs,notes,Date dernière ouverture,page,Série,Langue,total_read_time,total Nbr de pages lues,id long,Auteurs courts,format,categorie,oeuvre,First published date
67,68,Voyageur 2 : Libration,Becky Chambers,0,2024-10-03,504,Les voyageurs #2,fr,21135.0,487,00068,Becky Chambers,ebook,fiction/SF,Voyageur 2 : Libration - Becky Chambers,2022
68,69,Blackwater 3 – La Maison: L'épique saga de la ...,Michael McDowell,0,2024-10-08,213,Blackwater #3,fr,8572.0,206,00069,Michael McDowell,ebook,fiction/fantastique,Blackwater 3 – La Maison: L'épique saga de la ...,1983
69,70,Voyageur 3 : Archives de l'exode,Becky Chambers,0,2024-10-23,441,Les voyageurs #3,fr,22254.0,422,00070,Becky Chambers,ebook,fiction/SF,Voyageur 3 : Archives de l'exode - Becky Chambers,2020
70,71,Blackwater 4 – La Guerre: L'épique saga de la ...,Michael McDowell,0,2024-10-27,257,Blackwater #4,fr,8562.0,249,00071,Michael McDowell,ebook,fiction/fantastique,Blackwater 4 – La Guerre: L'épique saga de la ...,1983
71,72,Voyageur 4 : La Galaxie vue du sol,Becky Chambers,0,2024-11-01,400,Les voyageurs #4,fr,2951.0,54,00072,Becky Chambers,ebook,fiction/SF,Voyageur 4 : La Galaxie vue du sol - Becky Cha...,2021


In [56]:
df[colonne_ai] = ''

In [57]:
prompt_reviews = """
en utilisant tes connaissances, trouve les données suivantes pour chaque oeuvre : 
* community reviews (RatingStatistics__rating)
* nombre de notes (ratingsCount)
* nombre de reviews (reviewsCount)
* la date de mise à jour de ces 3 données (ou la date du jour si tu as trouvé l'information en direct)au format AAAA-MM

exemple pour l'oeuvre "A Psalm for the Wild-Built - Becky Chambers "
4.26|131901|23315|2024-11

N'ajoute rien d'autre. Utilise toujours le format donné en exemple

question: {oeuvre} 

catégorie:"""
prompt = PromptTemplate.from_template(prompt_reviews)

# langchain :
chain_prompt_reviews = (
    {"oeuvre": RunnablePassthrough()}
    | prompt
    | llm_gpt4o
    | StrOutputParser()
)

	
# fonction pour créer une colonne catégorie
def reviews(oeuvre):
    result = chain_prompt_reviews.invoke(oeuvre)
    return result


colonne_ai = "reviews"
colonne_ref = "oeuvre"

if colonne_ai not in df.columns:
    df[colonne_ai] = ''
df[colonne_ai] = df.apply(
    lambda row: reviews(row[colonne_ref]) if pd.isna(row[colonne_ai]) or row[colonne_ai] == '' else row[colonne_ai],
    axis=1
)
# print df[colonne_ai][0]
print(df[colonne_ai])


67                                4.34|1234|567|2023-10
68    Je suis désolé, mais je ne peux pas accéder à ...
69                                4.35|1234|567|2023-10
70    Je suis désolé, mais je ne peux pas fournir le...
71    Je suis désolé, mais je ne peux pas fournir le...
Name: reviews, dtype: object


In [49]:
df.head(20)

,id,Titre,Auteurs,notes,Date dernière ouverture,page,Série,Langue,total_read_time,total Nbr de pages lues,id long,Auteurs courts,format,categorie,oeuvre,First published date,reviews
67,68,Voyageur 2 : Libration,Becky Chambers,0,2024-10-03,504,Les voyageurs #2,fr,21135.0,487,00068,Becky Chambers,ebook,fiction/SF,Voyageur 2 : Libration - Becky Chambers,2022,"Je suis désolé, mais je ne peux pas accéder di..."
68,69,Blackwater 3 – La Maison: L'épique saga de la ...,Michael McDowell,0,2024-10-08,213,Blackwater #3,fr,8572.0,206,00069,Michael McDowell,ebook,fiction/fantastique,Blackwater 3 – La Maison: L'épique saga de la ...,1983,"Je suis désolé, mais je ne peux pas accéder di..."
69,70,Voyageur 3 : Archives de l'exode,Becky Chambers,0,2024-10-23,441,Les voyageurs #3,fr,22254.0,422,00070,Becky Chambers,ebook,fiction/SF,Voyageur 3 : Archives de l'exode - Becky Chambers,2020,"Je suis désolé, mais je ne peux pas accéder di..."
70,71,Blackwater 4 – La Guerre: L'épique saga de la ...,Michael McDowell,0,2024-10-27,257,Blackwater #4,fr,8562.0,249,00071,Michael McDowell,ebook,fiction/fantastique,Blackwater 4 – La Guerre: L'épique saga de la ...,1983,"Je suis désolé, mais je ne peux pas accéder di..."
71,72,Voyageur 4 : La Galaxie vue du sol,Becky Chambers,0,2024-11-01,400,Les voyageurs #4,fr,2951.0,54,00072,Becky Chambers,ebook,fiction/SF,Voyageur 4 : La Galaxie vue du sol - Becky Cha...,2021,"Je suis désolé, mais je ne peux pas accéder di..."


In [50]:
df.to_parquet('data_sources_from_python\df_annee.parquet', engine='pyarrow')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\chric\AppData\Local\Temp\ipykernel_188\3199247240.py:1: SyntaxWarning: invalid escape sequence '\d'
  df.to_parquet('data_sources_from_python\df_annee.parquet', engine='pyarrow')
